In [1]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="mesolitica/Classification-Speech-Instructions",
    repo_type='dataset',
    allow_patterns="data/*.parquet",
    local_dir="./Classification-Speech-Instructions",
)

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

'/home/mesolitica/stt/Classification-Speech-Instructions'

In [2]:
from glob import glob
from tqdm import tqdm
from multiprocess import Pool
import itertools
import zipfile
import os

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)


def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [3]:
from glob import glob
import pandas as pd
import os
import json
from transformers import AutoProcessor

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
tokenizer = processor.tokenizer

In [5]:
files = sorted(glob('Classification-Speech-Instructions/data/*.parquet'))
files

['Classification-Speech-Instructions/data/emotion-00000-of-00003.parquet',
 'Classification-Speech-Instructions/data/emotion-00001-of-00003.parquet',
 'Classification-Speech-Instructions/data/emotion-00002-of-00003.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00000-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00001-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00002-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00003-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00004-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00005-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part2-00006-of-00007.parquet',
 'Classification-Speech-Instructions/data/emotion_part3-00000-of-00001.parquet',
 'Classification-Speech-Instructions/data/gender_age-00000-of-00005.parquet',
 'Classification-Speech-Instructions/data/gender_

In [6]:
!mkdir Classification-Speech-Instructions-audio

In [7]:
def loop(files):
    files, _ = files
    data = []
    for f in files:
        f_only = os.path.split(f)[1].replace('.parquet', '')
        df = pd.read_parquet(f)
        for i in tqdm(range(len(df))):
            new_f = os.path.join('Classification-Speech-Instructions-audio', 
                                 f'{f_only}-{i}.mp3')
            with open(new_f, 'wb') as fopen:
                fopen.write(df['audio_filename'].iloc[i]['bytes'])
            
            try:
                conversation = [
                    {"role": "user", "content": [
                        {"type": "audio", "audio_url": "audio.wav"},
                        {"type": "text", "text": df['question'].iloc[i]},
                    ]},
                    {"role": "assistant", "content": df['answer'].iloc[i]},
                ]
                text = processor.apply_chat_template(conversation, tokenize=False)
            except Exception as e:
                continue
            
            data.append({
                'text': text,
                'audio': new_f,
            })
    return data

In [8]:
processed = loop((files[:1], 0))

100%|█████████████████████████████████████████████████████████████████████████████████| 3894/3894 [00:01<00:00, 2987.92it/s]


In [9]:
processed[-1]

{'text': "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nAudio 1: <|audio_bos|><|AUDIO|><|audio_eos|>\nCan you describe the emotional texture of this sound?<|im_end|>\n<|im_start|>assistant\nThe audio clip features a neutral tone as the word 'mob' is spoken. There's no particular emotional intensity detected in the delivery, suggesting a straightforward and unemotional recitation of the word.<|im_end|>\n",
 'audio': 'Classification-Speech-Instructions-audio/emotion-00000-of-00003-3893.mp3'}

In [10]:
import IPython.display as ipd
ipd.Audio(processed[-1]['audio']) 

In [11]:
processed = multiprocessing(files, loop, cores = min(len(files), 30))

100%|█████████████████████████████████████████████████████████████████████████████████| 9754/9754 [00:03<00:00, 3178.89it/s]


In [12]:
len(processed)

117508

In [13]:
with open('prepare-Classification-Speech-Instructions.json', 'w') as fopen:
    json.dump(processed, fopen)